# Защита персональных данных клиентов

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

**Цель исследования:** найти метод преобразования данных, с помощью которого будет сложно восстановить персональную информацию.

**Задачи исследования:**
- изучить данные
- умножить признаки на обратимую матрицу
- сравнить метрики по данным до и после преобразований
- предложить метод преобразования данных
- проверить метод

## Загрузка данных

**1. Загружу необходимые в работе библиотеки**

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import r2_score

**2. Загружу данные и изучу их**

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

В данных 5000 строк без пропусков. Столбцы "Пол", "Члены семьи", "Страховые выплаты" имеют формат целых чисел. Столбцы "Возраст" и "Зарплата" имеют формат вещественных чисел.

In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


**3. Сохраню features и target в переменные**

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

Целевой признак - страховые выплаты.

In [5]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

print(features.shape)
print(target.shape)

(5000, 4)
(5000,)


## Умножение матриц

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**1. Рассчитаю R2 до преобразований**

In [6]:
model = LinearRegression()
model.fit(features, target)
predicted = model.predict(features)
r2_before = r2_score(target, predicted)
print('R2:', r2_before)

R2: 0.4249455028666801


**2. Рассчитаю R2 после преобразований**

**2.1.** Создам рандомную обратимую матрицу

In [7]:
R = np.random.rand(4, 4)
R

array([[0.82095735, 0.87183218, 0.1863191 , 0.66241576],
       [0.99647959, 0.15590769, 0.84223533, 0.46762185],
       [0.35905532, 0.47584527, 0.44110848, 0.89470603],
       [0.37595158, 0.87427187, 0.64040704, 0.19023525]])

In [8]:
#проверим матрицу на обратимость

np.linalg.inv(R)

array([[ 1.01327486,  0.7593943 , -1.02170014, -0.58978206],
       [ 0.50090647, -0.66852345, -0.20784531,  0.87664238],
       [-1.26383558,  0.52927761,  0.46498878,  0.91284118],
       [-0.04994615, -0.21014688,  1.40899753, -0.67960168]])

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

Ошибки нет, матрица обратима.

**2.2.** Умножу признаки на матрицу и рассчитаю R2

In [9]:
features_after = features.values @ R

In [10]:
#обучим модель на новых признаках
model = LinearRegression()
model.fit(features_after, target)
predicted_after = model.predict(features_after)

#рассчитаем R2
r2_after = r2_score(target, predicted_after)

print('R2 после преобразований:', r2_after)

R2 после преобразований: 0.424945502866682


**3. Рассмотрю, как связаны параметры линейной регрессии в исходной задаче и в преобразованной**

In [11]:
#найдём отношение R2 до преобразований к R2 после преобразований

r2_before / r2_after

0.9999999999999956

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

Метрика практически не изменилась после преобразования.

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

**Ответ:** Метрика не меняется после преобразования.

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

**Обоснование:**

Вектор предсказаний до преобразования:

$$
a = Aw = A * (X^T * X)^{-1} * X^T * y
$$

Вектор предсказаний после преобразования:

$$
a_R = A * R * w_R = A * R * ((XR)^T * XR)^{-1} * (XR)^T * y
$$

Рассмотрю:
$$
((XR)^T * XR)^{-1} = (R^T*X^T*X*R)^{-1} = R^{-1}*(X^T*X)^{-1}*(R^T)^{-1}
$$

Подставлю:
$$
a_R = A*R*R^{-1}*(X^T*X)^{-1}*(R^T)^{-1}*(XR)^T*y = A*(X^T*X)^{-1}*(R^T)^{-1}*R^T*X^T*y = A*(X^T*X)^{-1}*X^T*y
$$

## Алгоритм преобразования

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

**Алгоритм**

1) сгенерирую рандомную матрицу
2) проверю её на обратимость
3) умножу данные на матрицу

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

**Обоснование**

1) сгенерирую рандомную матрицу\
Нужно сгенерировать матрицу с количеством строк, совпадающим с количеством столбцов матрицы признаков. Признаки - это данные пользователей.

2) проверю её на обратимость\
Нужна обратимая матрица, потому что при расчёте вектора весов используются обратные матрицы.

3) умножу данные на матрицу

## Проверка алгоритма

**1. Сгенерирую рандомную матрицу**

In [12]:
#сгенерируем матрицу размером 4Х4, т.к. признаков 4

R = np.random.rand(4, 4)
R

array([[0.43652927, 0.44529007, 0.21827682, 0.56144931],
       [0.06773307, 0.66007629, 0.62107911, 0.23155272],
       [0.62526481, 0.19700799, 0.40856175, 0.23592445],
       [0.04836355, 0.85243895, 0.56880366, 0.50732878]])

**2. Проверю её на обратимость**

In [13]:
np.linalg.inv(R)

array([[ 13.50572826,  18.68692546,  -8.33813172, -19.59798362],
       [ 25.68327041,  36.83250927, -19.10931218, -36.34756978],
       [-20.96195446, -27.20904105,  15.38039652,  28.46436344],
       [-20.93981035, -33.1631673 ,  15.6591899 ,  32.99887291]])

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

Ошибки нет, матрица обратима.

**3. Умножу данные на матрицу**

In [14]:
features_R = features @ R
print('Данные до преобразования:')
display(features.head())
print('Данные после преобразования:')
features_R.head()

Данные до преобразования:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


Данные после преобразования:


,0,1,2,3
0,31016.396685,9799.957336,20290.914351,11712.415382
1,23763.226987,7517.519702,15554.485117,8976.288024
2,13132.525337,4156.310076,8597.808140,4961.128573
3,26075.061833,8230.799810,17051.205434,9843.927017
4,16321.744680,5160.836057,10681.070286,6164.673188


**4. Проверю качество линейной регрессии**

In [15]:
model = LinearRegression()
model.fit(features, target)
print('R2 до преобразования:', model.score(features, target))

R2 до преобразования: 0.4249455028666801


In [16]:
model = LinearRegression()
model.fit(features_R, target)
print('R2 до преобразования:', model.score(features_R, target))

R2 до преобразования: 0.42494550286666033


<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

Качество линейной регрессии не поменялось.

<div class="alert alert-info" style="border:solid purple 2px; padding: 20px; background-color: #e6d2f2; color: #4b0082;">

**Итоговый вывод:**
- Я изучила данные и разделила их на признаки и целевой признак. В данных 5000 строк, 4 признака и один целевой признак.
- Сгенерировала рандомную матрицу и умножила на неё признаки. Обучила модели линейной регрессии на данных до и после преобразования и сравнила их качество метрикой R2. *Метрика не изменилась, значит, умножение данных на рандомную обратимую матрицу не меняет качество обучения.*


***Метод преобразования данных - умножение на рандомную обратимую матрицу.***